# 房贷

影响房贷的因素有哪些？

- 房产总价
- 房贷利率
- 首付金额
- 还款年限
- 房产升值速度
- 工资收入
- 理财收入
- 家庭生活支出

是否应该多付首付？是否应该尽可能贷款更多年限？

先说结论：
**如果稳定理财利率能够高于房贷贷款利率，那么首付少付、还款年限长合适；反之，如果稳定理财利率低于房贷贷款利率，那么首付多付、还款年限短合适。**


In [1]:
def calc_housing_loan(property_price, down_payment, mortgage_yearly_rate, mortgage_years, n_years):
    """
    计算 n_years 年后，(每月还款, 支付总额, 支付的利息总额, 剩余本金)
    """
    n_years = mortgage_years if n_years is None else n_years
    mortgage_monthly_rate = mortgage_yearly_rate / 12
    mortgage = property_price - down_payment

    # https://www.baike.com/wikiid/7153236291726755655?view_id=505dtqqrmfo00
    # 等额本息方式，每期应还款额 = 借款本金 × 月利率 × ((1＋月利率)的还款期数次方) / ((1＋月利率) 的 (还款期数 - 1) 次方)
    monthly_repayment = float(mortgage) * mortgage_monthly_rate * ((1 + mortgage_monthly_rate) ** (12 * mortgage_years)) / \
        ((1 + mortgage_monthly_rate) ** (12 * mortgage_years) - 1)

    # 计算 支付总额、支付的利息总额 和 剩余本金
    paid_period = n_years if n_years <= mortgage_years else mortgage_years
    paid_interest = 0
    remaining_principal = mortgage
    for x in range(0, paid_period * 12):
        current_interest = remaining_principal * mortgage_monthly_rate
        current_principal = monthly_repayment - current_interest
        paid_interest += current_interest
        remaining_principal -= current_principal

    total_paid = down_payment + monthly_repayment * paid_period * 12

    return (monthly_repayment, total_paid, paid_interest, remaining_principal)


def calc_cash_wealth(current_wealth, down_payment, monthly_repayment, mortgage_years, monthly_disposable_income,
                     return_on_investment, n_years):
    """
    计算 n_years 年后，现金价值
    """
    monthly_return_rate = return_on_investment / 12
    cash_wealth = current_wealth - down_payment
    for this_month in range(0, 12 * n_years):
        cash_wealth = cash_wealth * (1 + monthly_return_rate) + monthly_disposable_income
        if this_month <= mortgage_years * 12:
            cash_wealth = cash_wealth - monthly_repayment

    return cash_wealth


def calc_property_wealth(current_price, property_yearly_return, n_years):
    """
    计算 n_years 年后，房产价值
    """
    return current_price * ((1 + property_yearly_return)**n_years)


## 示例

假设小 Y 是一名在北京工作了五年的程序员，他看上了一套郊区小两居，价值 350 万元。小 Y 有 200 万可以动用的购房款。

问题：他是把 200 万全部用作首付，还是只交 35% 的首付款（122.5 万）？

假设：

- 房产的升值幅度：每年 3%
- 房贷利率：4.65%
- 工资税后收入：每月 3 万元
- 理财收入：按永久组合的收益率来算，是年化 6.8%
- 家庭生活支出：每月 5000 元

设置基础参数

In [2]:
# 单位：万元
current_wealth = 200
monthly_income = 3
family_expense = 0.5    # exclude repayment
monthly_disposable_income = monthly_income - family_expense

yearly_return_on_investment = 0.06
property_price = 350
property_yearly_return_rate = 0.03
mortgage_yearly_rate = 0.0465

wealth_years = 20


按揭贷款 30 年，首付 200 万

In [3]:
mortgage_years = 30
down_payment = 200

monthly_repayment, total_paid, interest_paid, remaining_principal = \
    calc_housing_loan(property_price, down_payment, mortgage_yearly_rate, mortgage_years, wealth_years)

cash_wealth = calc_cash_wealth(current_wealth, down_payment, monthly_repayment, mortgage_years,
                               monthly_disposable_income, yearly_return_on_investment, wealth_years)

property_wealth = calc_property_wealth(property_price, property_yearly_return_rate, wealth_years)

print("Mortgage Features:")
print("\tMortgage: %s Years, Down Payment: %s, Monthly Repayment: %.4f, Total Payment: %.2f" %
      (mortgage_years, down_payment, monthly_repayment, down_payment + mortgage_years * 12 * monthly_repayment))
print("%s Years later:" % wealth_years)
print("\tTotal Paid: %.2f, Paid Interest: %.2f, Remaining Principal: %.2f" %
      (total_paid, interest_paid, remaining_principal))
print("\tYour Cash wealth:\t", round(cash_wealth, 2))
print("\tYour Property wealth:\t", round(property_wealth, 2))


Mortgage Features:
	Mortgage: 30 Years, Down Payment: 200, Monthly Repayment: 0.7735, Total Payment: 478.44
20 Years later:
	Total Paid: 385.63, Paid Interest: 109.74, Remaining Principal: 74.11
	Your Cash wealth:	 797.73
	Your Property wealth:	 632.14


按揭贷款 30 年，首付 150 万

In [4]:
mortgage_years = 30
down_payment = 150

monthly_repayment, total_paid, interest_paid, remaining_principal = \
    calc_housing_loan(property_price, down_payment, mortgage_yearly_rate, mortgage_years, wealth_years)

cash_wealth = calc_cash_wealth(current_wealth, down_payment, monthly_repayment, mortgage_years,
                               monthly_disposable_income, yearly_return_on_investment, wealth_years)

property_wealth = calc_property_wealth(property_price, property_yearly_return_rate, wealth_years)

print("Mortgage Features:")
print("\tMortgage: %s Years, Down Payment: %s, Monthly Repayment: %.4f, Total Payment: %.2f" %
      (mortgage_years, down_payment, monthly_repayment, down_payment + mortgage_years * 12 * monthly_repayment))
print("%s Years later:" % wealth_years)
print("\tTotal Paid: %.2f, Paid Interest: %.2f, Remaining Principal: %.2f" %
      (total_paid, interest_paid, remaining_principal))
print("\tYour Cash wealth:\t", round(cash_wealth, 2))
print("\tYour Property wealth:\t", round(property_wealth, 2))

Mortgage Features:
	Mortgage: 30 Years, Down Payment: 150, Monthly Repayment: 1.0313, Total Payment: 521.26
20 Years later:
	Total Paid: 397.51, Paid Interest: 146.32, Remaining Principal: 98.82
	Your Cash wealth:	 844.12
	Your Property wealth:	 632.14


按揭贷款 20 年，首付 200 万

In [5]:
mortgage_years = 20
down_payment = 200

monthly_repayment, total_paid, interest_paid, remaining_principal = \
    calc_housing_loan(property_price, down_payment, mortgage_yearly_rate, mortgage_years, wealth_years)

cash_wealth = calc_cash_wealth(current_wealth, down_payment, monthly_repayment, mortgage_years,
                               monthly_disposable_income, yearly_return_on_investment, wealth_years)

property_wealth = calc_property_wealth(property_price, property_yearly_return_rate, wealth_years)

print("Mortgage Features:")
print("\tMortgage: %s Years, Down Payment: %s, Monthly Repayment: %.4f, Total Payment: %.2f" %
      (mortgage_years, down_payment, monthly_repayment, down_payment + mortgage_years * 12 * monthly_repayment))
print("%s Years later:" % wealth_years)
print("\tTotal Paid: %.2f, Paid Interest: %.2f, Remaining Principal: %.2f" %
      (total_paid, interest_paid, remaining_principal))
print("\tYour Cash wealth:\t", round(cash_wealth, 2))
print("\tYour Property wealth:\t", round(property_wealth, 2))

Mortgage Features:
	Mortgage: 20 Years, Down Payment: 200, Monthly Repayment: 0.9612, Total Payment: 430.68
20 Years later:
	Total Paid: 430.68, Paid Interest: 80.68, Remaining Principal: 0.00
	Your Cash wealth:	 711.01
	Your Property wealth:	 632.14


按揭贷款 20 年，首付 150 万

In [6]:
mortgage_years = 20
down_payment = 150

monthly_repayment, total_paid, interest_paid, remaining_principal = \
    calc_housing_loan(property_price, down_payment, mortgage_yearly_rate, mortgage_years, wealth_years)

cash_wealth = calc_cash_wealth(current_wealth, down_payment, monthly_repayment, mortgage_years,
                               monthly_disposable_income, yearly_return_on_investment, wealth_years)

property_wealth = calc_property_wealth(property_price, property_yearly_return_rate, wealth_years)

print("Mortgage Features:")
print("\tMortgage: %s Years, Down Payment: %s, Monthly Repayment: %.4f, Total Payment: %.2f" %
      (mortgage_years, down_payment, monthly_repayment, down_payment + mortgage_years * 12 * monthly_repayment))
print("%s Years later:" % wealth_years)
print("\tTotal Paid: %.2f, Paid Interest: %.2f, Remaining Principal: %.2f" %
      (total_paid, interest_paid, remaining_principal))
print("\tYour Cash wealth:\t", round(cash_wealth, 2))
print("\tYour Property wealth:\t", round(property_wealth, 2))

Mortgage Features:
	Mortgage: 20 Years, Down Payment: 150, Monthly Repayment: 1.2815, Total Payment: 457.57
20 Years later:
	Total Paid: 457.57, Paid Interest: 107.57, Remaining Principal: 0.00
	Your Cash wealth:	 728.48
	Your Property wealth:	 632.14


## 个人决策
—— 是否卖房 ？

In [7]:
# 单位：万元
current_wealth = 300
monthly_income = 0

yearly_return_on_investment = 0.07
property_price = 37
property_yearly_return_rate = 0.03
mortgage_yearly_rate = 0.0416

什么都不做

In [8]:
family_expense = 0.6
monthly_disposable_income = monthly_income - family_expense

mortgage_years = 20
down_payment = 10
wealth_years = 20

monthly_repayment, total_paid, interest_paid, remaining_principal = \
    calc_housing_loan(property_price, down_payment, mortgage_yearly_rate, mortgage_years, wealth_years)

cash_wealth = calc_cash_wealth(current_wealth, down_payment, monthly_repayment, mortgage_years,
                               monthly_disposable_income, yearly_return_on_investment, wealth_years)

property_wealth = calc_property_wealth(property_price, property_yearly_return_rate, wealth_years)

print("Mortgage Features:")
print("\tMortgage: %s Years, Down Payment: %s, Monthly Repayment: %.4f, Total Payment: %.2f" %
      (mortgage_years, down_payment, monthly_repayment, down_payment + mortgage_years * 12 * monthly_repayment))
print("%s Years later:" % wealth_years)
print("\tTotal Paid: %.2f, Paid Interest: %.2f, Remaining Principal: %.2f" %
      (total_paid, interest_paid, remaining_principal))
print("\tYour Cash wealth:\t", round(cash_wealth, 2))
print("\tYour Property wealth:\t", round(property_wealth, 2))

Mortgage Features:
	Mortgage: 20 Years, Down Payment: 10, Monthly Repayment: 0.1659, Total Payment: 49.82
20 Years later:
	Total Paid: 49.82, Paid Interest: 12.82, Remaining Principal: 0.00
	Your Cash wealth:	 772.26
	Your Property wealth:	 66.83


卖了房子，然后租房，每月租金2500

In [9]:
current_wealth = 450
monthly_income = 0
family_expense = 0.85
monthly_disposable_income = monthly_income - family_expense

mortgage_years = 1
down_payment = 10
wealth_years = 20

monthly_repayment, total_paid, interest_paid, remaining_principal = \
    calc_housing_loan(property_price, down_payment, mortgage_yearly_rate, mortgage_years, wealth_years)

cash_wealth = calc_cash_wealth(current_wealth, down_payment, monthly_repayment, mortgage_years,
                               monthly_disposable_income, yearly_return_on_investment, wealth_years)

property_wealth = 0

print("Mortgage Features:")
print("\tMortgage: %s Years, Down Payment: %s, Monthly Repayment: %.4f, Total Payment: %.2f" %
      (mortgage_years, down_payment, monthly_repayment, down_payment + mortgage_years * 12 * monthly_repayment))
print("%s Years later:" % wealth_years)
print("\tTotal Paid: %.2f, Paid Interest: %.2f, Remaining Principal: %.2f" %
      (total_paid, interest_paid, remaining_principal))
print("\tYour Cash wealth:\t", round(cash_wealth, 2))
print("\tYour Property wealth:\t", round(property_wealth, 2))

Mortgage Features:
	Mortgage: 1 Years, Down Payment: 10, Monthly Repayment: 2.3010, Total Payment: 37.61
20 Years later:
	Total Paid: 37.61, Paid Interest: 0.61, Remaining Principal: 0.00
	Your Cash wealth:	 1218.24
	Your Property wealth:	 0


卖了房子，换到乡间。乡间房屋一次性投入：购房25万和装修等费用25万

In [10]:
current_wealth = 400
monthly_income = 0
family_expense = 0.6
monthly_disposable_income = monthly_income - family_expense

mortgage_years = 1
down_payment = 10
wealth_years = 20

monthly_repayment, total_paid, interest_paid, remaining_principal = \
    calc_housing_loan(property_price, down_payment, mortgage_yearly_rate, mortgage_years, wealth_years)

cash_wealth = calc_cash_wealth(current_wealth, down_payment, monthly_repayment, mortgage_years,
                               monthly_disposable_income, yearly_return_on_investment, wealth_years)

property_wealth = calc_property_wealth(25, property_yearly_return_rate, wealth_years)

print("Mortgage Features:")
print("\tMortgage: %s Years, Down Payment: %s, Monthly Repayment: %.4f, Total Payment: %.2f" %
      (mortgage_years, down_payment, monthly_repayment, down_payment + mortgage_years * 12 * monthly_repayment))
print("%s Years later:" % wealth_years)
print("\tTotal Paid: %.2f, Paid Interest: %.2f, Remaining Principal: %.2f" %
      (total_paid, interest_paid, remaining_principal))
print("\tYour Cash wealth:\t", round(cash_wealth, 2))
print("\tYour Property wealth:\t", round(property_wealth, 2))

Mortgage Features:
	Mortgage: 1 Years, Down Payment: 10, Monthly Repayment: 2.3010, Total Payment: 37.61
20 Years later:
	Total Paid: 37.61, Paid Interest: 0.61, Remaining Principal: 0.00
	Your Cash wealth:	 1146.53
	Your Property wealth:	 45.15
